In [1]:
#!pip install nltk
#!conda install -c anaconda nltk

In [2]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
import string
from nltk.probability import FreqDist
import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer
import re
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

In [3]:
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('wordnet')

In [5]:
df = pd.read_csv('../Phase4_tweet_dataset.csv', encoding= 'unicode_escape')
df

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
...,...,...,...
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


In [7]:
df.dropna(inplace = True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3291 entries, 0 to 9088
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          3291 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  3291 non-null   object
dtypes: object(3)
memory usage: 102.8+ KB


In [9]:
## Confirming there arent any more null or unusable values
df['emotion_in_tweet_is_directed_at'].value_counts()

iPad                               946
Apple                              661
iPad or iPhone App                 470
Google                             430
iPhone                             297
Other Google product or service    293
Android App                         81
Android                             78
Other Apple product or service      35
Name: emotion_in_tweet_is_directed_at, dtype: int64

In [10]:
df['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

Positive emotion                      2672
Negative emotion                       519
No emotion toward brand or product      91
I can't tell                             9
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [11]:
sw = stopwords.words('english')
sw

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [12]:
#Using the Tweet Tokenizer to tokenize tweets
tweet_tokenizer = TweetTokenizer()

In [13]:
##not quite sure if we need but i think so
def get_wordnet_pos(treebank_tag):
    '''
    Translate nltk POS to wordnet tags
    '''
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [19]:
###syntax errors with sw, pos_tag not working within function (but works alone a few cells below) 
##tweet_tokenizer, .lower works
#also, train test split before running the function?
def doc_preparer(doc, stop_words=sw):
    '''
    
    :param doc: a document from the satire corpus 
    :return: a document string with words which have been 
            lemmatized, 
            parsed for stopwords, 
            made lowercase,
            and stripped of punctuation and numbers.
    '''
    
    tweet_tokenizer = TweetTokenizer()
    doc = df['tweet_text'].apply(tweet_tokenizer.tokenize)
    doc = [word.lower() for word in df['tweet_text']]
    doc = [word for word in df['tweet_text'] if word not in sw]
    # print(doc)
    doc = pos_tag(df['tweet_text'])
    doc = [(word[0], get_wordnet_pos(word[1])) for word in df['tweet_text']]
    lemmatizer = WordNetLemmatizer() 
    doc = [lemmatizer.lemmatize(word[0], word[1]) for word in df['tweet_text']]
    return ' '.join(doc)

In [21]:
#base code for our function above
def doc_preparer(doc, stop_words=sw):
    '''
    
    :param doc: a document from the satire corpus 
    :return: a document string with words which have been 
            lemmatized, 
            parsed for stopwords, 
            made lowercase,
            and stripped of punctuation and numbers.
    '''
    
    regex_token = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?)")
    doc = regex_token.tokenize(doc)
    doc = [word.lower() for word in doc]
    doc = [word for word in doc if word not in sw]
    # print(doc)
    doc = pos_tag(doc)
    doc = [(word[0], get_wordnet_pos(word[1])) for word in doc]
    lemmatizer = WordNetLemmatizer() 
    doc = [lemmatizer.lemmatize(word[0], word[1]) for word in doc]
    return ' '.join(doc)

In [20]:
doc

NameError: name 'doc' is not defined

In [15]:
##will be needed later
token_docs = [doc_preparer(doc, sw) for doc in X_train]
token_docs

NameError: name 'X_train' is not defined

In [17]:
doc1 = pos_tag(df['tweet_text'])
doc1

[('.@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead!  I need to upgrade. Plugin stations at #SXSW.',
  'JJ'),
 ("@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",
  'NNP'),
 ('@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.',
  'NNP'),
 ("@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw",
  'NNP'),
 ("@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",
  'NNP'),
 ('#SXSW is just starting, #CTIA is around the corner and #googleio is only a hop skip and a jump from there, good time to be an #android fan',
  'NNP'),
 ('Beautifully smart and simple idea RT @madebymany @thenextweb wrote about our #hollergram iPad app for #sxsw! http://bit.ly/ieaVOB',
  'NNP'),
 ('Counting down the days to #sxsw plus strong Canadian dollar m

In [22]:
##we will also need to add a punctuation removal to our function, this works well
df_cleaned = [s.translate(str.maketrans('', '', string.punctuation))\
                  for s in df['tweet_text']]
df_cleaned

['wesley83 I have a 3G iPhone After 3 hrs tweeting at RISEAustin it was dead  I need to upgrade Plugin stations at SXSW',
 'jessedee Know about fludapp  Awesome iPadiPhone app that youll likely appreciate for its design Also theyre giving free Ts at SXSW',
 'swonderlin Can not wait for iPad 2 also They should sale them down at SXSW',
 'sxsw I hope this years festival isnt as crashy as this years iPhone app sxsw',
 'sxtxstate great stuff on Fri SXSW Marissa Mayer Google Tim OReilly tech booksconferences amp Matt Mullenweg Wordpress',
 'SXSW is just starting CTIA is around the corner and googleio is only a hop skip and a jump from there good time to be an android fan',
 'Beautifully smart and simple idea RT madebymany thenextweb wrote about our hollergram iPad app for sxsw httpbitlyieaVOB',
 'Counting down the days to sxsw plus strong Canadian dollar means stock up on Apple gear',
 'Excited to meet the samsungmobileus at sxsw so I can show them my Sprint Galaxy S still running Android 21

In [23]:
##word frequency distribution, graph will only work after df{'tweet_text'] is fully preprocessed
fdist = FreqDist(tweet_tokens)
plt.figure(figsize=(10, 10))
fdist.plot(30);

NameError: name 'tweet_tokens' is not defined